In [1]:
# Import necessary packages

import pandas as pd
import ast
import json

In [2]:
# Load the dataset
df = pd.read_csv('../data/raw/spotify_playlists.csv')

# MAKE COMMENT

In [3]:
df['playlist_id'] = df['playlist_href'].apply(lambda x: x.split('/')[-1])
df['songs'] = df['songs'].apply(lambda x: ast.literal_eval(x))
# chatGPT helped here
df['songs'] = [songs + [{'playlist_id': playlist_id}] for songs, playlist_id in zip(df['songs'], df['playlist_id'])]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   playlist_href  23 non-null     object
 1   followers      23 non-null     int64 
 2   name           23 non-null     object
 3   description    23 non-null     object
 4   songs          23 non-null     object
 5   playlist_id    23 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.2+ KB


In [5]:
song = df['songs'][0][0]

song
with open('../data/raw/song.json', 'w') as f:
    json.dump(song, f)
#del song[['added_by', 'is_local', 'primary_color']]

In [6]:
# Copy was made to avoid SettingWithCopyWarning
playlists = df[['playlist_id', 'name', 'description']].copy()
playlists['num_followers'] = df['followers']
playlists['link'] = df['playlist_href']

In [12]:
def artist_list_to_string(artists):
    str = ""

    for artist in artists:
        str += artist['name'] + ", "

    str = str[:-2]
    return str
    
def clean_song_data(song: list, playlist_id):
    row = {}
    
    row['song_id'] = song['track']['external_ids']['isrc']
    row['playlist_id'] = playlist_id
    row['album_id'] = song['track']['album']['id']
    row['title'] = song['track']['name']
    row['release_date'] = song['track']['album']['release_date']
    row['release_date_precision'] = song['track']['album']['release_date_precision']
    row['explicit'] = song['track']['explicit']
    row['album_name'] = song['track']['album']['name']
    row['image'] = song['track']['album']['images'][2]['url']
    row['artists'] = artist_list_to_string(song['track']['artists'])
    row['popularity'] = song['track']['popularity']

    return row

After creating new functions we are ready to apply them to transform the data into a more managable format.

In [15]:
songs = df['songs'].apply(lambda song_list: [clean_song_data(song, song_list[-1]['playlist_id']) for song in song_list[:-1]])
songs = songs.explode().apply(pd.Series)